In [ ]:
# Requirements
!pip install tensorflow_datasets flax jax optax tqdm netket imgaug==0.2.6

In [4]:
import jax
import jax.numpy as jnp
import numpy as np
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import jax
import jax.numpy as jnp
import jax
import flax
import netket as nk
import jax.numpy as jnp
from jax.experimental import stax
import optax
from flax.training import train_state  # Useful dataclass to keep train state
from functools import partial
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/jax/experimental/stax.py:30: FutureWarning: jax.experimental.stax is deprecated, import jax.example_libraries.stax instead
  FutureWarning)


### Utility Functions

In [5]:
def show_img(img, ax=None, title=None):
  """Shows a single image.
  
  Must be stored as a 3d-tensor where the last dimension is 1 channel (greyscale)
  """
  if ax is None:
    ax = plt.gca()
  ax.imshow(img)
  ax.set_xticks([])
  ax.set_yticks([])
  if title:
    ax.set_title(title)

def show_img_grid(imgs, titles):
  """Shows a grid of images."""
  n = int(np.ceil(len(imgs)**.5))
  _, axs = plt.subplots(n, n, figsize=(3 * n, 3 * n))
  for i, (img, title) in enumerate(zip(imgs, titles)):
    show_img(img, axs[i // n][i % n], title)

## Setting up the dataset

In [6]:
#import
ds_builder = tfds.builder('cifar10')
ds_builder.download_and_prepare()
train_ds = tfds.as_numpy(ds_builder.as_dataset(split='train', batch_size=-1))
test_ds = tfds.as_numpy(ds_builder.as_dataset(split='test', batch_size=-1))

#delete the id 
del train_ds['id']
del test_ds['id']


# Normalize
train_ds['image'] = jnp.float32(train_ds['image']) / 255.
test_ds['image'] = jnp.float32(test_ds['image']) / 255.
print("dataset keys:", train_ds.keys())
print(f"The training dataset has shape: {train_ds['image'].shape} and dtype {train_ds['image'].dtype}")
print(f"The test     dataset has shape: {test_ds['image'].shape} and dtype {train_ds['image'].dtype}")
print("")
print(f"The training labels have shape: {train_ds['label'].shape} and dtype {train_ds['label'].dtype}")
print(f"The test     labels have shape: {test_ds['label'].shape} and dtype {test_ds['label'].dtype}")
print("The mean     of the data stored in the images are: ", np.mean(train_ds['image']))
print("The variance of the data stored in the images are: ", np.var(train_ds['image']))
#show_img_grid(
#    [train_ds['image'][idx] for idx in range(9)],
#    [f'label={train_ds["label"][idx]}' for idx in range(9)],
#)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incomplete9HNR88/cifar10-train.tfrecord


...

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incomplete9HNR88/cifar10-test.tfrecord


  0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


dataset keys: dict_keys(['image', 'label'])
The training dataset has shape: (50000, 32, 32, 3) and dtype float32
The test     dataset has shape: (10000, 32, 32, 3) and dtype float32

The training labels have shape: (50000,) and dtype int64
The test     labels have shape: (10000,) and dtype int64
The mean     of the data stored in the images are:  0.47336298
The variance of the data stored in the images are:  0.063286915


##Non-linearities

In [7]:
def modRelu(z, bias): # relu(|z|+b) * (z / |z|)
    norm = jnp.abs(z)
    scale = nk.nn.relu(norm + bias) / (norm + 1e-6)
    scaled = jax.lax.complex(jnp.real(z)*scale, jnp.imag(z)*scale)
    return scaled
def complex_relu(z):
    return jnp.where(z.real > 0, z, 0)
complex_relu = jax.jit(complex_relu)
modRelu=jax.jit(modRelu)

## Define Model

In [8]:
class Model(nk.nn.Module):
  n_classes : int = 10
  @nk.nn.compact
  def __call__(self, x, train):
    #make rng for dropout
    dropout_rng = self.make_rng('dropout')

    #first TWO convolutions 3x3 --> 32. Avgpol 2x2 stride 2x2. Dropout 0.2
    x = nk.nn.Conv(features=32, kernel_size=(3, 3))(x)
    x = nk.nn.relu(x)
    x = nk.nn.Conv(features=32, kernel_size=(3, 3))(x)
    x = nk.nn.relu(x)
    x = flax.linen.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = flax.linen.Dropout(0.2, deterministic=not train)(x)

    #second TWO convolutions 64x64 --> 32. Avgpol 2x2 stride 2x2. Dropout 0.2
    x = nk.nn.Conv(features=64, kernel_size=(3, 3))(x)
    x = nk.nn.relu(x)
    x = nk.nn.Conv(features=64, kernel_size=(3, 3))(x)
    x = nk.nn.relu(x)
    x = flax.linen.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = flax.linen.Dropout(0.2, deterministic=not train)(x)

    #third TWO convolutions 128x128 --> 32. Avgpol 2x2 stride 2x2. Dropout 0.2
    x = nk.nn.Conv(features=128, kernel_size=(3, 3))(x)
    x = nk.nn.relu(x)
    x = nk.nn.Conv(features=128, kernel_size=(3, 3))(x)
    x = nk.nn.relu(x)
    x = flax.linen.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = flax.linen.Dropout(0.2, deterministic=not train)(x)


    #flatten and dense 1
    x = x.reshape((x.shape[0], -1)) # Flatten
    x = nk.nn.Dense(features=128)(x)
    x = nk.nn.relu(x)

    #DROPOUT2
    #x = flax.linen.Dropout(0.5, deterministic=not train)(x) 

    #dense2 and softmax
    x = nk.nn.Dense(features=10)(x)
    x = nk.nn.log_softmax(x)
    return x


## Loss functions

In [9]:
# The loss function that we will use
def cross_entropy(*, logits, labels):
    one_hot_labels = jax.nn.one_hot(labels, num_classes=10)
    return -jnp.mean(jnp.sum(one_hot_labels * logits, axis=-1))

In [10]:
dropout_rng, init_dropout = jax.random.split(jax.random.PRNGKey(1))


In [11]:
def loss_fn(params, dropout_rng, images, labels):
    """
    Loss function minimised during training of the model.
    """
    logits = model.apply(params, images, rngs={'dropout' : dropout_rng}, train=True)
    return cross_entropy(logits=logits, labels=labels)

def compute_metrics(*, logits, labels):
    """
    Compute metrics of the model during training.
    
    Returns the loss and the accuracy.
    """
    loss = cross_entropy(logits=logits, labels=labels)
    accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)
    metrics = {
      'loss': loss,
      'accuracy': accuracy,
    }
    return metrics

## 4 - Create the setup and training loop

In [12]:
def create_train_state(example, rng, optimiser, dropout_rng):
    """Creates initial `TrainState`, holding the current parameters, state of the
    optimiser and other metadata.
    """
    # Construct the model parameters
    params = model.init({'params' : rng, 'dropout' : dropout_rng}, example, train=True)
        
    # Package all those informations in the model state
    return train_state.TrainState.create(
        apply_fn=model.apply, params=params, tx=optimiser)


In [13]:
@jax.jit
def eval_metrics(params, batch, dropout_rng):
    """
    This function evaluates the metrics without training the model.
    
    Used to check the performance of the network on training and test datasets.
    """
    logits = model.apply(params, batch['image'], rngs={'dropout' : dropout_rng}, train=False)
    return compute_metrics(logits=logits, labels=batch['label'])

In [14]:
def fgsm_update(image, data_grad, update_max_norm):
    """
    Compute the FGSM update on an image (or a batch of images)

    @param image: float32 tensor of shape (batch_size, rgb, height, width)
    @param data_grad: float32 tensor of the same shape as `image`. Gradient of the loss with respect to `image`.
    @param update_max_norm: float, the maximum permitted difference between `image` and the output of this function measured in L_inf norm.

    @returns a perturbed version of `image` with the same shape
    """
    # Collect the element-wise sign of the data gradient
    sign_data_grad = jnp.sign(data_grad)
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + update_max_norm*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = jnp.clip(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [15]:
def loss_fn2(images, params, dropout_rng, labels):
    """
    Loss function minimised during training of the model.
    """
    logits = model.apply(params, images, rngs={'dropout' : dropout_rng}, train=True)
    return cross_entropy(logits=logits, labels=labels)


In [16]:
@jax.jit
def train_step(state, batch, dropout_rng):
    """
    Train for a single step.
    
    """
    #Make parameters the only 'free' parameter
    _loss_fn = partial(loss_fn, dropout_rng = dropout_rng, images=batch['image'], labels=batch['label'])
    # construct the function returning the loss value and gradient.
    val_grad_fn = jax.value_and_grad(_loss_fn)
    # compute loss and gradient
    loss, grads = val_grad_fn(state.params)
    grads = jax.tree_map(lambda x: x.conj(), grads) 
    # update the state parameters 
    state = state.apply_gradients(grads=grads)

    # Generate adversarial examples
    _loss_fn2 = partial(loss_fn2, params=state.params, dropout_rng = dropout_rng, labels=batch['label'])
    grad = jax.grad(_loss_fn2)
    g = grad(batch['image'])
    #for epsilon in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    for epsilon in [8./255.]:
      new_images = fgsm_update(batch['image'], g, epsilon)
      # Train on these
      _loss_fn3 = partial(loss_fn, dropout_rng = dropout_rng, images=new_images, labels=batch['label'])
      val_grad_fn3 = jax.value_and_grad(_loss_fn3)
      loss, grads = val_grad_fn3(state.params)
      grads = jax.tree_map(lambda x: x.conj(), grads) 
      state = state.apply_gradients(grads=grads)
    # Evaluate the network again to get the log-probability distribution
    # over the batch images
    metrics = eval_metrics(state.params, batch, dropout_rng)
    
    return state, metrics

In [17]:
def train_epoch(state, train_ds, batch_size, epoch, rng, dropout_rng, *, max_steps=None):
    """Train for a single `epoch`.
    
    And epoch is composed of several steps, where every step is taken by updating
    the network parameters with a small mini-batch.
    """
    
    # total number of training images
    train_ds_size = len(train_ds['image'])
    
    # Compute how many steps are present in this epoch.
    steps_per_epoch = train_ds_size // batch_size

    # Truncate the number of steps
    if max_steps is not None:
        steps_per_epoch = min(steps_per_epoch, max_steps)

    # generate a random permutation of the indices to shuffle the training
    perms = jax.random.permutation(rng, train_ds_size)
    perms = perms[:steps_per_epoch * batch_size]
    perms = perms.reshape((steps_per_epoch, batch_size))
    
    # execute the training step for every mini-batch
    batch_metrics = []
    for perm in perms:
        batch = {k: v[perm, ...] for k, v in train_ds.items()}
        state, metrics = train_step(state, batch, dropout_rng)
        batch_metrics.append(metrics)

    # compute mean of metrics across each batch in epoch.
    batch_metrics_np = jax.device_get(batch_metrics)
    epoch_metrics_np = {
        k: np.mean([metrics[k] for metrics in batch_metrics_np])
            for k in batch_metrics_np[0]}

    return state, epoch_metrics_np


def evaluate_model(params, test_ds, dropout_rng):
    """
    evaluate the performance of the model on the test dataset
    """
    metrics = eval_metrics(params, test_ds, dropout_rng)
    metrics = jax.device_get(metrics)
    summary = jax.tree_map(lambda x: x.item(), metrics)
    return summary['loss'], summary['accuracy']

# 5 - Training model

In [23]:
models_saved = []
for s in range(5):
  # Definition of optimiser HyperParameters
  learning_rate = 0.005
  momentum = 0.9
  optimiser = optax.sgd(learning_rate, momentum)
  num_epochs = 80
  batch_size = 250
  max_steps = 200

  #define rngs
  seed = s #123
  seed_dropout = 10-s #0
  key = {'params': jax.random.PRNGKey(seed), 'dropout': jax.random.PRNGKey(seed_dropout)}

  #init model
  model = Model(n_classes=10)
  sample_input = jnp.ones([1, 32, 32, 3])
  pars = model.init(key, sample_input, train=True)

  # Split the rng to get two keys, one to 'shuffle' the dataset at every iteration,
  # and one to initialise the network
  rng, init_rng = jax.random.split(jax.random.PRNGKey(s))

  state = create_train_state(sample_input, init_rng, optimiser, init_dropout)
  metrics = {"test_loss" : [], "test_accuracy": [], "train_loss":[], "train_accuracy":[]}
  with tqdm(range(1, num_epochs + 1)) as pbar:
      for epoch in pbar:
          # Use a separate PRNG key to permute image data during shuffling
          rng, input_rng = jax.random.split(rng)
          dropout_rng, _ = jax.random.split(dropout_rng)
          # Run an optimization step over a training batch
          state, train_metrics = train_epoch(state, train_ds, batch_size, epoch, input_rng, dropout_rng)
          
          # Evaluate on the test set after each training epoch
          test_loss, test_accuracy = evaluate_model(state.params, test_ds, dropout_rng)
          pbar.write('train epoch: %d, loss: %.4f, accuracy: %.2f' % (epoch, train_metrics['loss'], train_metrics['accuracy'] * 100))
          pbar.write(' test epoch: %d, loss: %.2f, accuracy: %.2f' % (epoch, test_loss, test_accuracy * 100))

          # save data
          metrics["train_loss"].append(train_metrics["loss"])
          metrics["train_accuracy"].append(train_metrics["accuracy"])
          metrics["test_loss"].append(test_loss)
          metrics["test_accuracy"].append(test_accuracy)

  models_saved += [state]

    

  0%|          | 0/80 [00:00<?, ?it/s]

train epoch: 1, loss: 2.1143, accuracy: 21.78
 test epoch: 1, loss: 1.93, accuracy: 30.51
train epoch: 2, loss: 1.8213, accuracy: 34.90
 test epoch: 2, loss: 1.73, accuracy: 38.99
train epoch: 3, loss: 1.6671, accuracy: 40.48
 test epoch: 3, loss: 1.60, accuracy: 43.26
train epoch: 4, loss: 1.5748, accuracy: 43.81
 test epoch: 4, loss: 1.57, accuracy: 43.95
train epoch: 5, loss: 1.5072, accuracy: 46.50
 test epoch: 5, loss: 1.47, accuracy: 47.74
train epoch: 6, loss: 1.4446, accuracy: 49.25
 test epoch: 6, loss: 1.41, accuracy: 50.46
train epoch: 7, loss: 1.3958, accuracy: 50.88
 test epoch: 7, loss: 1.39, accuracy: 50.26
train epoch: 8, loss: 1.3527, accuracy: 52.48
 test epoch: 8, loss: 1.35, accuracy: 52.02
train epoch: 9, loss: 1.3067, accuracy: 54.47
 test epoch: 9, loss: 1.33, accuracy: 53.71
train epoch: 10, loss: 1.2793, accuracy: 55.67
 test epoch: 10, loss: 1.29, accuracy: 54.40
train epoch: 11, loss: 1.2481, accuracy: 56.87
 test epoch: 11, loss: 1.27, accuracy: 55.33
train 

  0%|          | 0/80 [00:00<?, ?it/s]

train epoch: 1, loss: 2.1262, accuracy: 20.66
 test epoch: 1, loss: 1.94, accuracy: 30.95
train epoch: 2, loss: 1.8411, accuracy: 34.41
 test epoch: 2, loss: 1.74, accuracy: 39.01
train epoch: 3, loss: 1.6795, accuracy: 40.11
 test epoch: 3, loss: 1.66, accuracy: 41.89
train epoch: 4, loss: 1.5876, accuracy: 43.99
 test epoch: 4, loss: 1.57, accuracy: 44.71
train epoch: 5, loss: 1.5228, accuracy: 46.13
 test epoch: 5, loss: 1.50, accuracy: 47.28
train epoch: 6, loss: 1.4765, accuracy: 48.10
 test epoch: 6, loss: 1.44, accuracy: 49.55
train epoch: 7, loss: 1.4299, accuracy: 49.60
 test epoch: 7, loss: 1.46, accuracy: 48.50
train epoch: 8, loss: 1.3960, accuracy: 51.01
 test epoch: 8, loss: 1.41, accuracy: 49.24
train epoch: 9, loss: 1.3604, accuracy: 52.43
 test epoch: 9, loss: 1.38, accuracy: 52.59
train epoch: 10, loss: 1.3277, accuracy: 53.81
 test epoch: 10, loss: 1.33, accuracy: 53.28
train epoch: 11, loss: 1.3015, accuracy: 54.73
 test epoch: 11, loss: 1.31, accuracy: 53.25
train 

  0%|          | 0/80 [00:00<?, ?it/s]

train epoch: 1, loss: 2.0881, accuracy: 22.81
 test epoch: 1, loss: 1.92, accuracy: 32.96
train epoch: 2, loss: 1.8148, accuracy: 35.33
 test epoch: 2, loss: 1.68, accuracy: 40.52
train epoch: 3, loss: 1.6530, accuracy: 41.33
 test epoch: 3, loss: 1.61, accuracy: 43.16
train epoch: 4, loss: 1.5618, accuracy: 44.77
 test epoch: 4, loss: 1.52, accuracy: 47.54
train epoch: 5, loss: 1.4939, accuracy: 47.01
 test epoch: 5, loss: 1.47, accuracy: 48.07
train epoch: 6, loss: 1.4339, accuracy: 49.32
 test epoch: 6, loss: 1.44, accuracy: 48.73
train epoch: 7, loss: 1.3857, accuracy: 51.26
 test epoch: 7, loss: 1.38, accuracy: 51.21
train epoch: 8, loss: 1.3492, accuracy: 52.71
 test epoch: 8, loss: 1.37, accuracy: 51.33
train epoch: 9, loss: 1.3124, accuracy: 54.32
 test epoch: 9, loss: 1.33, accuracy: 53.71
train epoch: 10, loss: 1.2832, accuracy: 55.78
 test epoch: 10, loss: 1.30, accuracy: 54.35
train epoch: 11, loss: 1.2557, accuracy: 56.65
 test epoch: 11, loss: 1.29, accuracy: 55.51
train 

  0%|          | 0/80 [00:00<?, ?it/s]

train epoch: 1, loss: 2.0825, accuracy: 23.29
 test epoch: 1, loss: 1.89, accuracy: 32.93
train epoch: 2, loss: 1.8186, accuracy: 35.48
 test epoch: 2, loss: 1.74, accuracy: 37.70
train epoch: 3, loss: 1.6468, accuracy: 41.55
 test epoch: 3, loss: 1.60, accuracy: 43.27
train epoch: 4, loss: 1.5624, accuracy: 44.81
 test epoch: 4, loss: 1.56, accuracy: 45.35
train epoch: 5, loss: 1.5051, accuracy: 46.77
 test epoch: 5, loss: 1.47, accuracy: 47.93
train epoch: 6, loss: 1.4505, accuracy: 48.93
 test epoch: 6, loss: 1.44, accuracy: 49.44
train epoch: 7, loss: 1.4033, accuracy: 50.59
 test epoch: 7, loss: 1.41, accuracy: 50.85
train epoch: 8, loss: 1.3624, accuracy: 52.02
 test epoch: 8, loss: 1.38, accuracy: 51.34
train epoch: 9, loss: 1.3306, accuracy: 53.63
 test epoch: 9, loss: 1.33, accuracy: 52.67
train epoch: 10, loss: 1.2891, accuracy: 55.12
 test epoch: 10, loss: 1.31, accuracy: 52.79
train epoch: 11, loss: 1.2655, accuracy: 56.11
 test epoch: 11, loss: 1.30, accuracy: 53.96
train 

In [27]:
import pickle
models_params = [m.params for m in models_saved]
with open("cifar_real_robust.txt", "wb") as fp:   #Pickling
  pickle.dump(models_params, fp)

In [17]:
#computing the number of parameters
tot_params = 0
for chiave in models_saved[0].params['params'].keys():
  for sotto_chiave in models_saved[0].params['params'][chiave]:
    print(chiave, ' \t', 
          sotto_chiave, '  \t', 
          models_saved[0].params['params'][chiave][sotto_chiave].size, '    \t', 
          models_saved[0].params['params'][chiave][sotto_chiave].dtype)
    tot_params += models_saved[0].params['params'][chiave][sotto_chiave].size
print('tot: ', tot_params)

Conv_0  	 bias   	 32     	 float64
Conv_0  	 kernel   	 864     	 float64
Conv_1  	 bias   	 32     	 float64
Conv_1  	 kernel   	 9216     	 float64
Conv_2  	 bias   	 64     	 float64
Conv_2  	 kernel   	 18432     	 float64
Conv_3  	 bias   	 64     	 float64
Conv_3  	 kernel   	 36864     	 float64
Conv_4  	 bias   	 128     	 float64
Conv_4  	 kernel   	 73728     	 float64
Conv_5  	 bias   	 128     	 float64
Conv_5  	 kernel   	 147456     	 float64
Dense_0  	 bias   	 128     	 float64
Dense_0  	 kernel   	 262144     	 float64
Dense_1  	 bias   	 10     	 float64
Dense_1  	 kernel   	 1280     	 float64
tot:  550570
